<p><img alt="NUS Logo" height="45px" src="https://drive.google.com/uc?export=view&id=1KqQfc9gfUT-LecSjYnpEQH9cG55UiBjS" align="left" hspace="10px" vspace="0px"></p>

## **IS5006 INTELLIGENT SYSTEMS DEPLOYMENT**
### **Individual Homework: Real Time Visualization of Financial Dataset (BTC/USDT) to PowerBI**
In this homework, python code and PowerBI (e.g. PowerBI API for streming dataset) will be used to build a dashboard with relevant charts and information.

For more information, please refer to IS5006 Week 7 Lecture Notes

Student ID: A0248330L

## **Installing Third-party Libraries**
To install necessary library e.g. ccxt (used toconnect and trade with cryptocurrency exchanges and payment processing services)

In [ ]:
!pip install ccxt

     |████████████████████████████████| 2.5 MB 8.5 MB/s 
     |████████████████████████████████| 271 kB 69.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.6 MB/s 
     |████████████████████████████████| 3.6 MB 41.5 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 291 kB 15.4 MB/s 
     |████████████████████████████████| 144 kB 51.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## **Importing the Necessary Libraries** 
To use the functions in modules and libraries, we have to first import them.

In [ ]:
import requests
import datetime
from datetime import datetime, timedelta
import json
import time
import ccxt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **Part 1: Realtime Arbitrage Calculation**

## **Declaring Constants**
To create/declare the "constants" (follow some standard conventions to emulate the semantic feel of constants as Python itself does not support non-changing value assignments in the way other languages that implement constants do) which will be used in the program.

In [ ]:
# for main() function
TRADING_COMISSION = 0.00075   #in percent

HITBTC_COMMISSION = 0.0025
BINANCE_COMMISSION = 0.0010

#put the url obtained from powerbi
URL = 'https://api.powerbi.com/beta/5ba5ef5e-3109-4e77-85bd-cfeb0d347e82/datasets/9c4a0ca3-c483-4c57-a5e5-ced21645f385/rows?key=Gz86dDY5nUJ4CxThcy6TM6w3TPjwLEPa8VEu7t3lILR49%2Bpf5d8YZVBjOCXwSnQfo1zVTV95fwBBMn%2B9oc2D1Q%3D%3D'

Adding values for trading so we can plot the amount of money made on arbitrage as well.

In [ ]:
CAPITAL = 10000.0

## **Adding Python User-defined Functions**
The following functions can be used to get the near real-time mid price for the BTC/USDT from both the HitBTC and Binance exhanges.

In [ ]:
#exchange hitbtc
def hitbtc_price():
  hitbtc = ccxt.hitbtc()
  # fetch the BTC/USDT ticker 
  bitcoin_hitbtc = hitbtc.fetch_ticker('BTC/USDT')
  #Getting the Mid price
  hitbtcResult = (float(bitcoin_hitbtc['info']['ask']) + float(bitcoin_hitbtc['info']['bid'])) / 2
  return hitbtcResult

#exchange binance
def binance_price():
  binance = ccxt.binance()
  # fetch the BTC/USDT ticker 
  bitcoin_binance = binance.fetch_ticker('BTC/USDT')
  #Getting the Mid price
  binance_price = (float(bitcoin_binance['ask']) + float(bitcoin_binance['bid'])) / 2
  return binance_price


The following main() function is used to detect if there is any arbitrage opportunities among the exchanges.

There is only an opportunity when the price difference exceeds the fees.

Adding profit calculation.

In [ ]:
def arbitrage_finder_two_markets():  
  hitprice = hitbtc_price()
  binance = binance_price()
  diff = hitprice-binance
  profit = None


  if diff > 0 and abs(diff) > TRADING_COMISSION*hitprice :
    profit = ((CAPITAL/binance)*hitprice - CAPITAL)/CAPITAL
    signal = "Sell at hitbtc, Buy at binance"
  elif diff < 0 and abs(diff) > TRADING_COMISSION*binance :
    profit = ((CAPITAL/hitprice)*binance - CAPITAL)/CAPITAL
    signal = "Buy at hitbtc, Sell at binance"
  else:
    profit = 0
    signal ="No opportunity"

  return hitprice, binance, diff, signal, profit

## **Final Step: Main Caller Function**
Main caller function that starts the process (keep looping and read the latest prices from the exchanges) and displays the result to console.

**This also sends the arbitrage finder data to PowerBI dashboard.**

In [ ]:
headers = {"Content-Type": "application/json"}

In [ ]:
if __name__ == "__main__":
    while True:
      now = datetime.strftime(datetime.now(),"%Y-%m-%d %H:%M:%S")
      hitprice, binance, diff, signal, profit = arbitrage_finder_two_markets()
      print(f'Time: {now}, Return: {profit:.4f} Hitbtc_Price : {hitprice:.2f}, Binance_Price :  {binance:.2f}, Difference : {abs(diff):.2f}, Arbitrage : {signal}')
      json_data = [{
        "Hitbtc_Price" :hitprice,
        "Binance_Price" :binance,
        "Difference" :abs(diff),
        "Arbitrage" :signal,
        "Return": profit,
        "Time" :now 
      }]

      #Send data to PowerBI Here
      response = requests.request(
        method="POST",
        url=URL,
        headers=headers,
        data=json.dumps(json_data)
      )
      
      # wait for 2 seconds before next loop
      time.sleep(2)

Time: 2022-03-09 18:10:44, Return: 0.0000 Hitbtc_Price : 42229.55, Binance_Price :  42233.99, Difference : 4.44, Arbitrage : No opportunity
Time: 2022-03-09 18:10:48, Return: 0.0000 Hitbtc_Price : 42225.29, Binance_Price :  42233.43, Difference : 8.14, Arbitrage : No opportunity
Time: 2022-03-09 18:10:52, Return: 0.0000 Hitbtc_Price : 42224.07, Binance_Price :  42233.43, Difference : 9.36, Arbitrage : No opportunity
Time: 2022-03-09 18:10:56, Return: 0.0000 Hitbtc_Price : 42227.42, Binance_Price :  42237.40, Difference : 9.99, Arbitrage : No opportunity
Time: 2022-03-09 18:10:59, Return: 0.0000 Hitbtc_Price : 42222.69, Binance_Price :  42229.76, Difference : 7.07, Arbitrage : No opportunity
Time: 2022-03-09 18:11:04, Return: 0.0000 Hitbtc_Price : 42235.87, Binance_Price :  42238.06, Difference : 2.19, Arbitrage : No opportunity
Time: 2022-03-09 18:11:08, Return: 0.0000 Hitbtc_Price : 42222.49, Binance_Price :  42231.38, Difference : 8.89, Arbitrage : No opportunity
Time: 2022-03-09 18:

KeyboardInterrupt: ignored

# Part 2: Historical Data Trade

We get Historical Data from both exchanges from 2019 to end of 2021 to develop trading strategies and evaluate.

## Collect exchange data

- Defining constants to use the ccxt api to pull prices
- Iterating over date ranges to work around limit restriction on getting prices 

In [ ]:
# Defining constants
exchanges = ['hitbtc', 'binance']
period = '1d'
start = datetime(2018, 10, 31) # Earlier date to facilitate moving average calculation
end = datetime(2022, 1, 1)
limit = 1000
days = (end - start).days
symbol = 'BTC/USDT'

In [ ]:
df = pd.DataFrame(columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Exchange'])
for exch in exchanges:
  exchange = getattr(ccxt, exch)()
  exchange.load_markets()
  for i in range(int(days//limit)+1):
    since = int((start+timedelta(limit*i)).timestamp())*1000
    history = exchange.fetch_ohlcv(symbol, period, since, limit)
    temp_df = pd.DataFrame(history)
    temp_df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    temp_df['Exchange'] = exch
    temp_df['Timestamp'] = pd.to_datetime(temp_df['Timestamp'], unit='ms')
    df = df.append(temp_df)
df.head()

,Timestamp,Open,High,Low,Close,Volume,Exchange
0,2018-10-31,6331.56,6421.35,6250.99,6371.44,7006.39,hitbtc
1,2018-11-01,6371.25,6427.84,6357.01,6408.83,8608.14,hitbtc
2,2018-11-02,6405.93,6449.73,6394.07,6432.05,9725.60,hitbtc
3,2018-11-03,6432.61,6435.76,6346.92,6388.88,8883.29,hitbtc
4,2018-11-04,6391.43,6522.97,6358.15,6480.99,7992.05,hitbtc


Merge binance and hitbtc dataframes side by side to facilitate further calculations.

In [ ]:
merged_df = df[df['Exchange']=='binance'].merge(df[df['Exchange']=='hitbtc'], on='Timestamp', suffixes = ['_binance', '_hitbtc'])
merged_df.drop(['Exchange_binance', 'Exchange_hitbtc'], axis=1, inplace = True)
merged_df.set_index('Timestamp', inplace=True)
merged_df.head()

,Open_binance,High_binance,Low_binance,Close_binance,Volume_binance,Open_hitbtc,High_hitbtc,Low_hitbtc,Close_hitbtc,Volume_hitbtc
Timestamp,,,,,,,,,,
2018-10-31,6330.01,6428.00,6245.02,6371.93,12148.888216,6331.56,6421.35,6250.99,6371.44,7006.39
2018-11-01,6369.52,6442.65,6348.66,6410.00,9099.035841,6371.25,6427.84,6357.01,6408.83,8608.14
2018-11-02,6410.00,6460.34,6388.20,6433.98,9739.440679,6405.93,6449.73,6394.07,6432.05,9725.60
2018-11-03,6432.80,6439.97,6345.00,6387.09,7661.241476,6432.61,6435.76,6346.92,6388.88,8883.29
2018-11-04,6388.00,6525.00,6359.00,6485.85,10592.394943,6391.43,6522.97,6358.15,6480.99,7992.05


## Trade Strategies

Building trade strategies using prices for the two exchanges

- Defining constants fro trading
- Using 3 different approaches

In [ ]:
# Starting Capital
CAPITAL = 10000.0
BTC = 0.0

price = 'Close'

In [ ]:
signal_df = merged_df.copy(deep = True)
signal_df.head()

,Open_binance,High_binance,Low_binance,Close_binance,Volume_binance,Open_hitbtc,High_hitbtc,Low_hitbtc,Close_hitbtc,Volume_hitbtc
Timestamp,,,,,,,,,,
2018-10-31,6330.01,6428.00,6245.02,6371.93,12148.888216,6331.56,6421.35,6250.99,6371.44,7006.39
2018-11-01,6369.52,6442.65,6348.66,6410.00,9099.035841,6371.25,6427.84,6357.01,6408.83,8608.14
2018-11-02,6410.00,6460.34,6388.20,6433.98,9739.440679,6405.93,6449.73,6394.07,6432.05,9725.60
2018-11-03,6432.80,6439.97,6345.00,6387.09,7661.241476,6432.61,6435.76,6346.92,6388.88,8883.29
2018-11-04,6388.00,6525.00,6359.00,6485.85,10592.394943,6391.43,6522.97,6358.15,6480.99,7992.05


### Moving Average Crossover

We generate a strategy using Moving Average Crossover by creating signals when a slower moving average crosses a faster moving average.

- Exponential Moving Average is used for the faster MA
- Simple Moving Average is used for the slower MA

When generating the signal
- 1 indicates Buy
- -1 indicates Sell
- 0 indicates no trade

In [ ]:
for exch in exchanges:
  signal_df[f'EMA_10_{exch}'] = signal_df[f'{price}_{exch}'].ewm(span=10, adjust = False).mean()
  signal_df[f'SMA_50_{exch}'] = signal_df[f'{price}_{exch}'].rolling(50).mean()
  signal_df[f'MA_Position_{exch}'] = np.where(signal_df[f'EMA_10_{exch}'] > signal_df[f'SMA_50_{exch}'], 1.0, 0.0)
  signal_df[f'MA_{exch}_Signal'] = signal_df[f'MA_Position_{exch}'].diff()
  signal_df.drop([f'MA_Position_{exch}'], axis=1, inplace=True)

In [ ]:
signal_df.head()

,Open_binance,High_binance,Low_binance,Close_binance,Volume_binance,Open_hitbtc,High_hitbtc,Low_hitbtc,Close_hitbtc,Volume_hitbtc,EMA_10_hitbtc,SMA_50_hitbtc,MA_hitbtc_Signal,EMA_10_binance,SMA_50_binance,MA_binance_Signal
Timestamp,,,,,,,,,,,,,,,,
2018-10-31,6330.01,6428.00,6245.02,6371.93,12148.888216,6331.56,6421.35,6250.99,6371.44,7006.39,6371.440000,NaN,NaN,6371.930000,NaN,NaN
2018-11-01,6369.52,6442.65,6348.66,6410.00,9099.035841,6371.25,6427.84,6357.01,6408.83,8608.14,6378.238182,NaN,0.0,6378.851818,NaN,0.0
2018-11-02,6410.00,6460.34,6388.20,6433.98,9739.440679,6405.93,6449.73,6394.07,6432.05,9725.60,6388.022149,NaN,0.0,6388.875124,NaN,0.0
2018-11-03,6432.80,6439.97,6345.00,6387.09,7661.241476,6432.61,6435.76,6346.92,6388.88,8883.29,6388.178122,NaN,0.0,6388.550556,NaN,0.0
2018-11-04,6388.00,6525.00,6359.00,6485.85,10592.394943,6391.43,6522.97,6358.15,6480.99,7992.05,6405.053009,NaN,0.0,6406.241364,NaN,0.0


### Arbitrage Strategy

Using arbitrage strategy as before on historical data

We use values 0,1,-1 in the arbitrage strategy to stay consistent with how we have indicated the signal from the moving average strategy.

Thus:

- A signal of -1 indicates buy from hitbtc and sell on binance
- A signal of 1 indicates buy from binance and sell on hitbtc
- A signal of 0 means perform no trade

In [ ]:
# Define trading commission in percentage
TRADING_COMISSION = 0.00075

In [ ]:
# Function to calculate Signal 
def arbitrage_signal(row):
  hitbtc = row[f'{price}_hitbtc']
  binance = row[f'{price}_binance']
  diff = hitbtc - binance
  if diff > 0 and abs(diff) > TRADING_COMISSION*hitbtc:
    return 1
  elif diff < 0 and abs(diff) > TRADING_COMISSION*binance:
    return -1
  else: 
    return 0

In [ ]:
signal_df['Arbitrage_Signal'] = signal_df.apply(lambda row: arbitrage_signal(row), axis=1)

In [ ]:
signal_df.head()

,Open_binance,High_binance,Low_binance,Close_binance,Volume_binance,Open_hitbtc,High_hitbtc,Low_hitbtc,Close_hitbtc,Volume_hitbtc,EMA_10_hitbtc,SMA_50_hitbtc,MA_hitbtc_Signal,EMA_10_binance,SMA_50_binance,MA_binance_Signal,Arbitrage_Signal
Timestamp,,,,,,,,,,,,,,,,,
2018-10-31,6330.01,6428.00,6245.02,6371.93,12148.888216,6331.56,6421.35,6250.99,6371.44,7006.39,6371.440000,NaN,NaN,6371.930000,NaN,NaN,0
2018-11-01,6369.52,6442.65,6348.66,6410.00,9099.035841,6371.25,6427.84,6357.01,6408.83,8608.14,6378.238182,NaN,0.0,6378.851818,NaN,0.0,0
2018-11-02,6410.00,6460.34,6388.20,6433.98,9739.440679,6405.93,6449.73,6394.07,6432.05,9725.60,6388.022149,NaN,0.0,6388.875124,NaN,0.0,0
2018-11-03,6432.80,6439.97,6345.00,6387.09,7661.241476,6432.61,6435.76,6346.92,6388.88,8883.29,6388.178122,NaN,0.0,6388.550556,NaN,0.0,0
2018-11-04,6388.00,6525.00,6359.00,6485.85,10592.394943,6391.43,6522.97,6358.15,6480.99,7992.05,6405.053009,NaN,0.0,6406.241364,NaN,0.0,0


### Naive Holding Strategy

This strategy incorporates buying on 1 January 2019 and selling at current date.


When generating the signal
- 1 indicates Buy
- -1 indicates Sell
- 0 indicates no trade

In [ ]:
buy_date = '2019-01-01'
sell_date = signal_df.index[-1]

Timestamp('2022-03-10 00:00:00')

In [ ]:
signal_df['Naive_hitbtc_Signal'] = 0
signal_df['Naive_binance_Signal'] = 0

In [ ]:
signal_df.at['2019-01-01', 'Naive_hitbtc_Signal'] = 1
signal_df.at['2019-01-01', 'Naive_binance_Signal'] = 1
signal_df.at[sell_date, 'Naive_hitbtc_Signal'] = -1
signal_df.at[sell_date, 'Naive_binance_Signal'] = -1

In [ ]:
signal_df[signal_df['Naive_hitbtc_Signal'] != 0]

,Open_binance,High_binance,Low_binance,Close_binance,Volume_binance,Open_hitbtc,High_hitbtc,Low_hitbtc,Close_hitbtc,Volume_hitbtc,EMA_10_hitbtc,SMA_50_hitbtc,MA_hitbtc_Signal,EMA_10_binance,SMA_50_binance,MA_binance_Signal,Arbitrage_Signal,Naive_hitbtc_Signal,Naive_binance_Signal
Timestamp,,,,,,,,,,,,,,,,,,,
2019-01-01,3701.23,3810.16,3642.00,3797.14,23741.687033,3720.44,3822.67,3663.83,3811.44,21810.58000,3792.582131,4137.603,0.0,3756.673438,4078.7016,0.0,1,1,1
2022-03-10,41941.70,42039.63,38848.48,39234.15,29164.146950,41953.14,42036.95,38900.18,39237.30,11489.54838,39926.817387,39891.614,0.0,39923.639833,39889.0072,0.0,0,-1,-1


## Calculate PnL 

Using the dignal from our 3 strategies we calculate PnL for our trades

We follow the buy and sell signals as defined in our strategies and make trades using all money available when we are to buy and al bitcoin available when we are to sell.

In [ ]:
trade_df = signal_df.copy(deep = True)
trade_df = trade_df[trade_df.index > datetime(2018, 12, 31)]
trade_df.head()

,Open_binance,High_binance,Low_binance,Close_binance,Volume_binance,Open_hitbtc,High_hitbtc,Low_hitbtc,Close_hitbtc,Volume_hitbtc,EMA_10_hitbtc,SMA_50_hitbtc,MA_hitbtc_Signal,EMA_10_binance,SMA_50_binance,MA_binance_Signal,Arbitrage_Signal,Naive_hitbtc_Signal,Naive_binance_Signal
Timestamp,,,,,,,,,,,,,,,,,,,
2019-01-01,3701.23,3810.16,3642.00,3797.14,23741.687033,3720.44,3822.67,3663.83,3811.44,21810.58,3792.582131,4137.6030,0.0,3756.673438,4078.7016,0.0,1,1,1
2019-01-02,3796.45,3882.14,3750.45,3858.56,35156.463369,3811.62,3955.60,3767.04,3868.13,33110.63,3806.318107,4085.8540,0.0,3775.198267,4026.7196,0.0,1,0,0
2019-01-03,3857.57,3862.74,3730.00,3766.78,29406.948359,3867.79,3871.73,3770.00,3775.35,9605.38,3800.687542,4042.7060,0.0,3773.667673,3983.6070,0.0,1,0,0
2019-01-04,3767.20,3823.64,3703.57,3792.01,29519.554671,3779.78,3827.09,3712.31,3792.06,37402.23,3799.118898,4003.6062,0.0,3777.002642,3944.3792,0.0,0,0,0
2019-01-05,3790.09,3840.99,3751.00,3770.96,30490.667751,3794.56,3844.27,3773.78,3801.54,8562.48,3799.559099,3966.3280,0.0,3775.903980,3906.6796,0.0,1,0,0


In [ ]:
signals = ['MA_binance_Signal', 'MA_hitbtc_Signal', 'Arbitrage_Signal', 'Naive_hitbtc_Signal', 'Naive_binance_Signal']

After listing out the different signals we have, we iterate over the rows of the dataframe to do a rolling calculation on the proft/loss we'd make on each trade and calculate how our portfolio increases.

In [ ]:
for signal in signals:
  trade_df[f'{signal}_pnl'] = 0.0
  capital = CAPITAL
  if(signal == 'Arbitrage_Signal'):
    for i,row in trade_df.iterrows():
      if(row[signal] == 1):
        capital = capital/row[f'{price}_binance']*(row[f'{price}_hitbtc']*(1-(0.01*TRADING_COMISSION)))
      elif(row[signal] == -1):
        capital = capital/row[f'{price}_hitbtc']*(row[f'{price}_binance']*(1-(0.01*TRADING_COMISSION)))
      else:
        pass
      trade_df.at[i, f'{signal}_pnl'] = capital
  else:
    exch = signal.split('_')[1]
    btc = BTC
    for i, row in trade_df.iterrows():
      if(row[signal] == 1):
        btc = capital/row[f'{price}_{exch}']
        capital = 0
      elif(row[signal] == -1):
        capital = btc*row[f'{price}_{exch}']
        btc = 0
      else:
        pass
      curr_pnl = capital + (btc*row[f'{price}_{exch}'])
      trade_df.at[i, f'{signal}_pnl'] = curr_pnl

In [ ]:
trade_df.tail()

,Open_binance,High_binance,Low_binance,Close_binance,Volume_binance,Open_hitbtc,High_hitbtc,Low_hitbtc,Close_hitbtc,Volume_hitbtc,...,SMA_50_binance,MA_binance_Signal,Arbitrage_Signal,Naive_hitbtc_Signal,Naive_binance_Signal,MA_binance_Signal_pnl,MA_hitbtc_Signal_pnl,Naive_hitbtc_Signal_pnl,Naive_binance_Signal_pnl,Arbitrage_Signal_pnl
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-03-06,39397.97,39693.87,38088.57,38420.81,39677.26158,39378.11,39710.00,38110.19,38421.87,14205.91536,...,40116.8256,0.0,0,0,0,108248.512022,104835.243237,100806.702978,101183.548671,12309.493174
2022-03-07,38420.80,39547.57,37155.00,37988.00,63941.20316,38417.88,39549.98,37160.00,38007.30,21351.29805,...,40015.1524,-1.0,0,0,0,107029.093730,103704.076358,99719.003841,100043.717113,12309.493174
2022-03-08,37988.01,39362.08,37867.65,38730.63,55528.43367,37997.01,39360.00,37883.70,38730.13,19473.54487,...,39945.7326,0.0,0,0,0,107029.093730,103704.076358,101615.478664,101999.478555,12309.493174
2022-03-09,38730.63,42594.06,38656.45,41941.71,67392.58799,38735.80,42551.99,38643.19,41950.36,20294.76975,...,39937.5244,1.0,0,0,0,107029.093730,103704.076358,110064.332641,110456.053767,12309.493174
2022-03-10,41941.70,42039.63,38848.48,39234.15,29164.14695,41953.14,42036.95,38900.18,39237.30,11489.54838,...,39889.0072,0.0,0,-1,-1,100119.797638,96997.211830,102946.130596,103325.529214,12309.493174


## Save results

Save results to file for further visualization in PowerBI

In [ ]:
trade_df.to_csv('Historical_PnL.csv')